# Costumer Segmentation - RFM Analysis

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import locale
from statistics import mean
import scipy.stats as stats
import glob as glob
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

## Importing data

In [2]:
# Using glob function to grab the name of all the files that start and finish with specified string
files = glob.glob('table_billing_*.xlsx')
# Effectively reading the tables with pandas
df_list = [pd.read_excel(files[i]) for i in range(len(files))]

In [3]:
# Concating the tables into a single dataframe
df = pd.concat(df_list)

# Renaming the columns
df.columns = ['MATERIAL_COD', 'MATERIAL_NOME', 'LINHA_PRODUTO',
       'STOCK_ORDER', 'CLIENTE_CORP_ID', 'CLIENTE_CORP_NOME',
       'CLIENTE_COMPLETO_ID', 'CLIENTE_COMPLETO_NOME', 'CLIENTE_COMPLETO_CIDADE', 'CLIENTE_COMPLETO_UF',
       'CLIENTE_COMPLETO_CEP', 'SEGMENTO', 'CANAL', 'MERCADO',
       'DATA', 'PERIODO', 'UNIDADES', 'CUSTO',
       'VENDAS', 'RESULTADO']

In [4]:
# Setting the locale variable
locale.setlocale(locale.LC_ALL, 'pt-BR.UTF-8')

# Cleaning the data with bad readings
# The UNIDADES column accounts for peaces sold and the RESULTADO column is the revenue from that sale
# If both are less or equal to zero, we don't want that
df = df.drop(df[(df.UNIDADES==0) & (df.RESULTADO==0)].index)
# Cleaning bad date readings
df = df[df['DATA']>='2019-01-01']

# Creating copies to be used
df2 = df.copy()

## Setting time windows

In [5]:
# Grabbing the first date of the dataframe and setting to the first they of said month
first_date = df['DATA'].min().replace(day=1)
# Grabbing the date of one year ago
year_ago = (datetime.today().date().replace(day=1) - relativedelta(months=12))

# Calculating the number of months between those two dates
periods = (year_ago.year - first_date.year)*12 + year_ago.month - first_date.month +1

# Generating a list of inferior limits
dates_inf = pd.date_range(start=first_date, periods=periods, freq='MS')
dates_inf

DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
               '2021-01-01', '2021-02-01'],
              dtype='datetime64[ns]', freq='MS')

In [6]:
# Grabbing the first date and adding 12 months 
first_date_sup = dates_inf[0] + relativedelta(months=12) - relativedelta(days=1)

# Generating a list of superior limits
dates_sup = pd.date_range(start=first_date_sup, periods=periods, freq='M')
dates_sup

DatetimeIndex(['2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31',
               '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31',
               '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
               '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
               '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31',
               '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
               '2021-12-31', '2022-01-31'],
              dtype='datetime64[ns]', freq='M')

### Example

For purposes of better understanding I am going to break the for loop to show the outputs of each step

In [7]:
# First we sellect a time window of 12 months of purchase history
# The windows are set from the ranges created above
df = df2.loc[(df2['DATA']>=dates_inf[1]) & (df2['DATA']<=dates_sup[1])]

df[['MATERIAL_COD', 'MATERIAL_NOME', 'LINHA_PRODUTO', 'CLIENTE_CORP_ID', 'CLIENTE_COMPLETO_ID', 
    'DATA','UNIDADES', 'CUSTO', 'VENDAS', 'RESULTADO']].head(3)

,MATERIAL_COD,MATERIAL_NOME,LINHA_PRODUTO,CLIENTE_CORP_ID,CLIENTE_COMPLETO_ID,DATA,UNIDADES,CUSTO,VENDAS,RESULTADO
1,5539500001,SHS SST 225x275 CAR01 CAR C60,Coated,20244980,20244980,2019-02-05,500.0,119.197,175.84,56.643
7,5539500001,SHS SST 225x275 CAR01 CAR C60,Coated,20479654,20479654,2019-02-26,500.0,119.197,150.81,31.613
8,5539500001,SHS SST 225x275 CAR01 CAR C60,Coated,20479946,20479946,2019-02-27,500.0,119.197,174.52,55.323


In [8]:
# Getting the list of all clients in the dataframe
clientes = pd.DataFrame(df2.CLIENTE_CORP_ID.unique())
# Renaming the columns to ID
clientes.columns = ['ID']
clientes['PERIOD'] = dates_sup[1]
clientes.head(3)   

,ID,PERIOD
0,20244030,2020-01-31
1,20244980,2020-01-31
2,20275231,2020-01-31


In [9]:
# Grabbing all dates each client made a purchase
dfn = df[['CLIENTE_CORP_ID', 'DATA']].drop_duplicates()
dfn.head(3)

,CLIENTE_CORP_ID,DATA
1,20244980,2019-02-05
7,20479654,2019-02-26
8,20479946,2019-02-27


## Recency

Recency measures the time interval between the client's last purschase and the present.

In [10]:
# Grouping the dataframe by the clients and grabbing the biggest date of each one
# This will give us the last date each client has made a purchase
last_purchase = dfn.groupby('CLIENTE_CORP_ID').DATA.max().reset_index()
# Renaming columns
last_purchase.columns = ['ID', 'ULTIMA_COMPRA']
# The recency will be the time interval between the last purchase by the client and the current date
# Since we are analyzing over a moving window the current date will be the superior limit
last_purchase['RECENCY']=(dates_sup[1]-last_purchase['ULTIMA_COMPRA']).dt.days
# Merging the information to out clients dataframe
clientes = pd.merge(clientes, last_purchase[['ID','RECENCY']], on='ID')
clientes.head(3)

,ID,PERIOD,RECENCY
0,20244030,2020-01-31,29
1,20244980,2020-01-31,304
2,20275231,2020-01-31,31


## Frequency

Frequency measures how many times a client has purchased in a given time interval.

In [11]:
# Grouping the dataframe by the clients and grabbing a count of dates
# This will give us how many times each client has made a purchase
frequency = dfn.groupby('CLIENTE_CORP_ID').DATA.count().reset_index()
# Renaming columns
frequency.columns = ['ID', 'FREQUENCY']
# Merging the information to out clients dataframe
clientes = pd.merge(clientes, frequency, on='ID')
clientes.head(3)

,ID,PERIOD,RECENCY,FREQUENCY
0,20244030,2020-01-31,29,9
1,20244980,2020-01-31,304,2
2,20275231,2020-01-31,31,13


## Monetary

Monetary measures how much a client has spent in a given time interval.

In [12]:
# Grouping the dataframe by the clients and grabbing the sum of sales result
# This will give us how much each client has spent
revenue = df.groupby('CLIENTE_CORP_ID').RESULTADO.sum().reset_index()
# Renaming columns
revenue.columns = ['ID', 'MONETARY']
# Merging the information to out clients dataframe
clientes = pd.merge(clientes, revenue, on='ID')
clientes.head(3)

,ID,PERIOD,RECENCY,FREQUENCY,MONETARY
0,20244030,2020-01-31,29,9,38204.633380
1,20244980,2020-01-31,304,2,1098.156300
2,20275231,2020-01-31,31,13,101123.843082


## Quintile function

In [13]:
# This function is used to rank each metric
def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).values.argmax()
    return series.rank(pct=1).apply(f)

In [14]:
# Using qcut to discretize based on the metrics quintiles
# Every metric will be ranked from 1 to 5 
clientes['RECENCY_QUINTILE'] = pd.qcut(clientes['RECENCY'], 5, labels=False)
clientes['MONETARY_QUINTILE'] = pd.qcut(clientes['MONETARY'], 5, labels=False)
clientes['FREQUENCY_QUINTILE'] = pd.qcut(clientes['FREQUENCY'], 5, labels=False)

# Adjusting the metrics (Recency is up side down and the other are 0 to 4)
clientes['RECENCY_QUINTILE'] = abs(clientes['RECENCY_QUINTILE'] -5)
clientes['MONETARY_QUINTILE'] = clientes['MONETARY_QUINTILE'] +1
clientes['FREQUENCY_QUINTILE'] = clientes['FREQUENCY_QUINTILE'] +1

clientes.head(3)

,ID,PERIOD,RECENCY,FREQUENCY,MONETARY,RECENCY_QUINTILE,MONETARY_QUINTILE,FREQUENCY_QUINTILE
0,20244030,2020-01-31,29,9,38204.633380,4,5,4
1,20244980,2020-01-31,304,2,1098.156300,1,1,1
2,20275231,2020-01-31,31,13,101123.843082,4,5,5


In [15]:
# Taking the mean of monetary and frequency
clientes['MEDIA_F_M'] = (clientes['MONETARY_QUINTILE'] + clientes['FREQUENCY_QUINTILE']) /2

# Classificating the clients based on their scores

# LOST
clientes.loc[((clientes['RECENCY_QUINTILE']<=2) & (clientes['MEDIA_F_M']<=2)), 'CLASS']= 'LOST'
# HIBERNATING
clientes.loc[(((clientes['RECENCY_QUINTILE']<=2) & (clientes['RECENCY_QUINTILE']>1)) & ((clientes['MEDIA_F_M']<=2) & (clientes['MEDIA_F_M']>1))), 'CLASS']= 'HIBERNATING'
# DANGER
clientes.loc[(((clientes['RECENCY_QUINTILE']<=2) & (clientes['RECENCY_QUINTILE']>0)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>2))), 'CLASS']= 'DANGER'
# DONT_LOSE
clientes.loc[(((clientes['RECENCY_QUINTILE']<=1) & (clientes['RECENCY_QUINTILE']>0)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>=4))), 'CLASS']= 'DONT_LOSE'
# LOYAL
clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>2)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>3))), 'CLASS']= 'LOYAL'
# CHAMPION
clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>4)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>4))), 'CLASS']= 'CHAMPION'
# ABOUTO_SLEEP
clientes.loc[(((clientes['RECENCY_QUINTILE']<=3) & (clientes['RECENCY_QUINTILE']>2)) & ((clientes['MEDIA_F_M']<=2) & (clientes['MEDIA_F_M']>0))), 'CLASS']= 'ABOUTO_SLEEP'
# ATTENCION
clientes.loc[(((clientes['RECENCY_QUINTILE']<=3) & (clientes['RECENCY_QUINTILE']>2)) & ((clientes['MEDIA_F_M']<=3) & (clientes['MEDIA_F_M']>2))), 'CLASS']= 'ATTENCION'
# POTENCIAL
clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>3)) & ((clientes['MEDIA_F_M']<=3) & (clientes['MEDIA_F_M']>1))), 'CLASS']= 'POTENCIAL'
#PROMISING
clientes.loc[(((clientes['RECENCY_QUINTILE']<=4) & (clientes['RECENCY_QUINTILE']>3)) & ((clientes['MEDIA_F_M']<=1) & (clientes['MEDIA_F_M']>0))), 'CLASS']= 'PROMISING'
# NEWBIES
clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>4)) & ((clientes['MEDIA_F_M']<=1) & (clientes['MEDIA_F_M']>0))), 'CLASS']= 'NEWBIES'

# Getting the important columns
clientes_segmentado = clientes[['ID', 'PERIOD', 'CLASS','RECENCY_QUINTILE','FREQUENCY_QUINTILE','MONETARY_QUINTILE']]
clientes_segmentado.head(3)

,ID,PERIOD,CLASS,RECENCY_QUINTILE,FREQUENCY_QUINTILE,MONETARY_QUINTILE
0,20244030,2020-01-31,LOYAL,4,4,5
1,20244980,2020-01-31,LOST,1,1,1
2,20275231,2020-01-31,LOYAL,4,5,5


### Running for all clients

In [16]:
dfinal = pd.DataFrame()

for n in range(len(dates_sup)):
    # First we sellect a time window of 12 months of purchase history
    # The windows are set from the ranges created above
    df = df2.loc[(df2['DATA']>=dates_inf[n]) & (df2['DATA']<=dates_sup[n])]
    
    # Getting the list of all clients in the dataframe
    clientes = pd.DataFrame(df2.CLIENTE_CORP_ID.unique())
    # Renaming the columns to ID
    clientes.columns = ['ID']
    clientes['PERIOD'] = dates_sup[n]
    
    # Grabbing all dates each client made a purchase
    dfn = df[['CLIENTE_CORP_ID', 'DATA']].drop_duplicates()
    
    #Recency
    # Grouping the dataframe by the clients and grabbing the biggest date of each one
    # This will give us the last date each client has made a purchase
    last_purchase = dfn.groupby('CLIENTE_CORP_ID').DATA.max().reset_index()
    # Renaming columns
    last_purchase.columns = ['ID', 'ULTIMA_COMPRA']
    # The recency will be the time interval between the last purchase by the client and the current date
    # Since we are analyzing over a moving window the current date will be the superior limit
    last_purchase['RECENCY']=(dates_sup[n]-last_purchase['ULTIMA_COMPRA']).dt.days
    # Merging the information to out clients dataframe
    clientes = pd.merge(clientes, last_purchase[['ID','RECENCY']], on='ID')
    
    #Frequency
    # Grouping the dataframe by the clients and grabbing a count of dates
    # This will give us how many times each client has made a purchase
    frequency = dfn.groupby('CLIENTE_CORP_ID').DATA.count().reset_index()
    # Renaming columns
    frequency.columns = ['ID', 'FREQUENCY']
    # Merging the information to out clients dataframe
    clientes = pd.merge(clientes, frequency, on='ID')
    
    #Monetary
    # Grouping the dataframe by the clients and grabbing the sum of sales result
    # This will give us how much each client has spent
    revenue = df.groupby('CLIENTE_CORP_ID').RESULTADO.sum().reset_index()
    # Renaming columns
    revenue.columns = ['ID', 'MONETARY']
    # Merging the information to out clients dataframe
    clientes = pd.merge(clientes, revenue, on='ID')
    
    
    # Using qcut to discretize based on the metrics quintiles
    # Every metric will be ranked from n to 5 
    clientes['RECENCY_QUINTILE'] = pd.qcut(clientes['RECENCY'], 5, labels=False)
    clientes['FREQUENCY_QUINTILE'] = pct_rank_qcut(clientes.FREQUENCY, 5)
    clientes['MONETARY_QUINTILE'] = pd.qcut(clientes['MONETARY'], 5, labels=False)

    # Adjusting the metrics (Recency is up side down and the other are 0 to 4)
    clientes['RECENCY_QUINTILE'] = abs(clientes['RECENCY_QUINTILE'] -5)
    clientes['MONETARY_QUINTILE'] = clientes['MONETARY_QUINTILE'] +1

    
    
    clientes['MEDIA_F_M'] = (clientes['MONETARY_QUINTILE'] + clientes['FREQUENCY_QUINTILE']) /2


    # Classificating the clients based on their scores

    # LOST
    clientes.loc[((clientes['RECENCY_QUINTILE']<=2) & (clientes['MEDIA_F_M']<=2)), 'CLASS']= 'LOST'
    # HIBERNATING
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=2) & (clientes['RECENCY_QUINTILE']>1)) & ((clientes['MEDIA_F_M']<=2) & (clientes['MEDIA_F_M']>1))), 'CLASS']= 'HIBERNATING'
    # DANGER
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=2) & (clientes['RECENCY_QUINTILE']>0)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>2))), 'CLASS']= 'DANGER'
    # DONT_LOSE
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=1) & (clientes['RECENCY_QUINTILE']>0)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>=4))), 'CLASS']= 'DONT_LOSE'
    # LOYAL
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>2)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>3))), 'CLASS']= 'LOYAL'
    # CHAMPION
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>4)) & ((clientes['MEDIA_F_M']<=5) & (clientes['MEDIA_F_M']>4))), 'CLASS']= 'CHAMPION'
    # ABOUTO_SLEEP
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=3) & (clientes['RECENCY_QUINTILE']>2)) & ((clientes['MEDIA_F_M']<=2) & (clientes['MEDIA_F_M']>0))), 'CLASS']= 'ABOUTO_SLEEP'
    # ATTENCION
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=3) & (clientes['RECENCY_QUINTILE']>2)) & ((clientes['MEDIA_F_M']<=3) & (clientes['MEDIA_F_M']>2))), 'CLASS']= 'ATTENCION'
    # POTENCIAL
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>3)) & ((clientes['MEDIA_F_M']<=3) & (clientes['MEDIA_F_M']>1))), 'CLASS']= 'POTENCIAL'
    #PROMISING
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=4) & (clientes['RECENCY_QUINTILE']>3)) & ((clientes['MEDIA_F_M']<=1) & (clientes['MEDIA_F_M']>0))), 'CLASS']= 'PROMISING'
    # NEWBIES
    clientes.loc[(((clientes['RECENCY_QUINTILE']<=5) & (clientes['RECENCY_QUINTILE']>4)) & ((clientes['MEDIA_F_M']<=1) & (clientes['MEDIA_F_M']>0))), 'CLASS']= 'NEWBIES'


    clientes_segmentado = clientes[['ID', 'PERIOD', 'CLASS','RECENCY_QUINTILE','FREQUENCY_QUINTILE','MONETARY_QUINTILE']]
    
    dfinal = dfinal.append(clientes_segmentado).reset_index(drop=True)
    
dfinal.tail(3)

,ID,PERIOD,CLASS,RECENCY_QUINTILE,FREQUENCY_QUINTILE,MONETARY_QUINTILE
210411,20240103,2022-01-31,LOST,1,1,1
210412,20240122,2022-01-31,ABOUTO_SLEEP,3,1,2
210413,20266294,2022-01-31,LOST,1,1,1


## Plotting real examples

In [17]:
costumer = '20479946'
x = dfinal[dfinal['ID']==costumer].PERIOD

trace1 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].RECENCY_QUINTILE,
                name = 'RECENCY',
                text = dfinal[dfinal['ID']==costumer].CLASS)
trace2 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].FREQUENCY_QUINTILE,
                name = 'FREQUENCY',
                text = dfinal[dfinal['ID']==costumer].CLASS)
trace3 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].MONETARY_QUINTILE,
                name = 'MONETARY',
                text = dfinal[dfinal['ID']==costumer].CLASS)

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'CLIENT 1 <br> RFM Score',
                   title_x = 0.5,
                   yaxis = {'title': 'Score', 'range': [0, 5]},
                   xaxis = {'range' : ['2019-12-01', '2021-06-01']})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [20]:
costumer = '20494305'
x = dfinal[dfinal['ID']==costumer].PERIOD

trace1 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].RECENCY_QUINTILE,
                name = 'RECENCY',
                text = dfinal[dfinal['ID']==costumer].CLASS)
trace2 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].FREQUENCY_QUINTILE,
                name = 'FREQUENCY',
                text = dfinal[dfinal['ID']==costumer].CLASS)
trace3 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].MONETARY_QUINTILE,
                name = 'MONETARY',
                text = dfinal[dfinal['ID']==costumer].CLASS)

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'CLIENT 2 <br> RFM Score',
                   title_x = 0.5)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [21]:
costumer = '20461548'
x = dfinal[dfinal['ID']==costumer].PERIOD

trace1 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].RECENCY_QUINTILE,
                name = 'RECENCY',
                text = dfinal[dfinal['ID']==costumer].CLASS)
trace2 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].FREQUENCY_QUINTILE,
                name = 'FREQUENCY',
                text = dfinal[dfinal['ID']==costumer].CLASS)
trace3 = go.Bar(x = x,
                y = dfinal[dfinal['ID']==costumer].MONETARY_QUINTILE,
                name = 'MONETARY',
                text = dfinal[dfinal['ID']==costumer].CLASS)

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'CLIENT 3 <br> RFM Score',
                   title_x = 0.5)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)